In [27]:
import numpy as np
def compactState(yard):
    sort = []
    for stack in yard:
      for container in stack:
        if not container in sort:
          sort.append(container)
    sort = sorted(sort)
    maxValue = 0
    for i in range(len(yard)):
      for j in range(len(yard[i])):
        yard[i][j] = sort.index(yard[i][j]) + 1
        if yard[i][j] > maxValue:
          maxValue = yard[i][j]
    return yard

def elevateState(yard, h, max_item):
    for stack in yard:
      while len(stack) < h:
        stack.insert(0,2*max_item)
    return yard

def flattenState(state):
    flatten = []
    for lista in state:
        for item in lista:
            flatten.append(item)
    return flatten

def normalize(state,max_item):
    return np.array(state)/max_item


def getStackValues(yard): #sorted stacks?
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if i==0:
                cont += 1
            else:
                if stack[i] <= stack[i-1]:
                    cont += 1
                else: break
        values.append(cont)
    return values

def getStackLen(yard):
    lens = []
    for stack in yard:
        lens.append(len(stack))
    return lens

def getTopStacks(yard,max_item):
    tops = []
    for stack in yard:
        if len(stack) != 0:
            tops.append(stack[len(stack)-1])
        else:
            tops.append(max_item)
    return tops

In [28]:
import copy
def generate_ann_state(yard,H):
    yard=copy.deepcopy(yard)
    max_item = max(set().union(*yard))
    #max_item = max([max(stack) for stack in yard])
    stackValues=getStackValues(yard) #well-placed
    stacksLen = getStackLen(yard)
    topStacks = getTopStacks(yard,max_item)
    yard=compactState(yard)
    yard=elevateState(yard,H, max_item)
    yard=flattenState(yard)
    yard=normalize(yard,max_item)
    state=yard
    #state=np.append(state,normalize(stackValues,max_item))
    #state=np.append(state,normalize(stacksLen,max_item))
    #state=np.append(state,normalize(topStacks,max_item))
    return state

In [29]:
import copy
import Greedy as gr
import Layout

def generate_ann_state_stack(layout, s, H):
    layout=copy.deepcopy(layout)
    max_item=max(set().union(*layout.stacks))
    
    #SF variables
    sf_possible = gr.SF_move_(layout)[0]!=False
    is_sorted = layout.is_sorted_stack(s)
    
    min_dif = max_item+1
    for i in range(len(layout.stacks)):
        if i==s or len(layout.stacks[s])==0: continue
        c = layout.stacks[s][-1]
        dif = Layout.gvalue(layout.stacks[i]) - c
        
        if layout.is_sorted_stack(i) and len(layout.stacks[i]) < H and (dif >= 0) and (dif < min_dif):
            min_dif=dif    
    min_dif/=max_item     
    top = Layout.gvalue(layout.stacks[s])/max_item
    
    #SD variables
    size = len(layout.stacks[s])/H
    mean = np.mean(layout.stacks[s])/max_item
    rh = Layout.reachable_height(layout, s)/H
    sd_active = len(layout.moves)>0  and (layout.prev!="bg" and layout.moves[-1][0]==s)
    
    return np.array([sf_possible,is_sorted,min_dif,top,size,mean,rh,sd_active])


In [30]:
import Pixie as px
layout = px.read("Instancias/CVS/3-3/data3-3-5.dat")
print(layout.stacks)
state=generate_ann_state_stack(layout, 0, 5)
state
#state.shape=(5,7)
#state

[[9, 3, 2], [5, 6, 1], [8, 7, 4]]


array([1.        , 1.        , 0.22222222, 0.22222222, 0.6       ,
       0.51851852, 1.        , 0.        ])

In [31]:
from __future__ import print_function
import os
import neat
import visualize
import copy
from Greedy import greedy_solve
import Layout

layouts = []
layouts.append(px.read("Instancias/CVS/5-5/data5-5-1.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-2.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-3.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-4.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-6.dat"))


def generate_initial_layouts():
    global layouts
    return copy.deepcopy(layouts)

def eval_genomes(genomes, config):
    
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        layouts=generate_initial_layouts()
        
        genome.fitness=0.0
        
        for layout in layouts: 
            best_ev=-1.0
            lay_copy = copy.deepcopy(layout)
            max_steps = greedy_solve(lay_copy, max_steps=100)

            for i in range(int(1.2*max_steps)):
                max_value=-np.inf
                for s in range(len(layout.stacks)):
                    output = net.activate(generate_ann_state_stack(layout, s, 7))
                    value = output[0]
                    if value>max_value: 
                        max_value=value
                        s_o=s

                s_d,_=Layout.select_destination_stack(layout, s_o)
                if s_d==None: break 
                layout.move(s_o,s_d)
                
                lay_copy = copy.deepcopy(layout)
                steps = greedy_solve(lay_copy, max_steps=1.2*max_steps)
                if steps == None: break

                ev = -float(steps)/float(max_steps) + 0.01*i
                if ev > best_ev: best_ev=ev
            genome.fitness+=best_ev
            
        genome.fitness/=5.0
        

In [ ]:
from __future__ import print_function
import os
import neat
import visualize



def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(50))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 3000)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    #visualize.draw_net(config, winner, True, node_names=node_names)
    #visualize.plot_stats(stats, ylog=False, view=True)
    #visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)



# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
local_dir = os.path.dirname(".")
config_path = os.path.join(local_dir, 'neat-config.txt')
run(config_path)

In [32]:
p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-325')


In [33]:
eval_genomes(p.population.items(), p.config)

C:\Users\Dell\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Dell\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
max_ev=-1.0
for gen in p.population:
    if p.population[gen].fitness > max_ev: 
        max_ev=p.population[gen].fitness
        genome=p.population[gen]
max_ev

-0.6142633053221287

In [54]:
evals=np.zeros(40)
evalss = []
for k in range(10,41):
    layout = px.read("Instancias/CVS/5-10/data5-10-"+str(k)+".dat")
    
    #print(genome.fitness)
    net = neat.nn.FeedForwardNetwork.create(genome, p.config)

    lay_copy = copy.deepcopy(layout)
    max_steps = greedy_solve(lay_copy, max_steps=100)
    if max_steps==None: max_steps=100 
    min_steps = max_steps
    best_i=-1
    print(max_steps)
    for i in range(max_steps):  
        max_value=-np.inf
        vals = []
        for s in range(len(layout.stacks)):
            output = net.activate(generate_ann_state_stack(layout, s, 7))
            value = output[0]
            vals.append(value)
            if value>max_value: 
                max_value=value
                s_o=s

        s_d,_=Layout.select_destination_stack(layout, s_o)
        if s_d==None: continue 
        layout.move(s_o,s_d)

        lay_copy = copy.deepcopy(layout)
        steps = greedy_solve(lay_copy, max_steps=1.2*max_steps)
        if steps==None: continue
        print(vals, "\n", layout.stacks, i+1, steps)
        #print(layout.stacks,steps)
        if steps<min_steps:
            best_i=i+1
            min_steps=steps

        if steps>1.2*max_steps: break
  

    print(max_steps,min_steps,best_i)
    break
    
    if max_steps<100: evalss.append(max_steps)
    evals[k-1]=min_steps
print(np.mean(evals) )
print(np.mean(evalss) )

58
[8.990562642643096e-25, 9.01295858664698e-25, 8.928347017715642e-25, 9.138751601417048e-25, 9.077624442914007e-25, 9.033664068939055e-25, 9.022032286684324e-25, 8.733990249672611e-25, 8.95908094198384e-25, 8.928347017715642e-25] 
 [[4, 15, 29, 47, 30, 32], [18, 28, 27, 20, 39], [3, 42, 48, 13, 2], [44, 50, 21, 40], [26, 46, 31, 43, 10], [38, 49, 9, 35, 8], [33, 41, 14, 23, 24], [11, 25, 22, 5, 6], [34, 45, 1, 17, 19], [36, 16, 12, 7, 37]] 1 56
[8.92010472476357e-25, 9.01295858664698e-25, 8.928347017715642e-25, 4.40083268528998e-24, 9.077624442914007e-25, 9.033664068939055e-25, 9.022032286684324e-25, 8.733990249672611e-25, 8.95908094198384e-25, 8.928347017715642e-25] 
 [[4, 15, 29, 47, 30, 32], [18, 28, 27, 20, 39, 40], [3, 42, 48, 13, 2], [44, 50, 21], [26, 46, 31, 43, 10], [38, 49, 9, 35, 8], [33, 41, 14, 23, 24], [11, 25, 22, 5, 6], [34, 45, 1, 17, 19], [36, 16, 12, 7, 37]] 2 56
[8.92010472476357e-25, 8.95608489159357e-25, 8.928347017715642e-25, 2.0926339539670444e-14, 9.077624442

C:\Users\Dell\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Dell\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


![image](https://i.imgur.com/s7kq3Md.png)